In [1]:
import os,sys,random, pickle
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="1" 
from keras import regularizers, metrics
from keras.models import Model
# noinspection PyPep8Naming
from keras import backend as K
from keras import optimizers, callbacks, losses
from keras.layers import * #Input, Softmax, Embedding, Add, Lambda, Dense, Multiply, Bidirectional, CuDNNGRU, CuDNNLSTM,LSTM,BatchNormalization,Concatenate
from keras.activations import hard_sigmoid, sigmoid
from keras.utils.training_utils import multi_gpu_model

from keras.models import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from keras.engine.topology import *
from keras.optimizers import *
import numpy as np
import pandas as pd
from sklearn.metrics import *

from keras_transformer.extras import ReusableEmbedding, TiedOutputEmbedding
from keras_transformer.position import TransformerCoordinateEmbedding
from keras_transformer.transformer import TransformerACT, TransformerBlock
from keras_transformer.bert import (
    BatchGeneratorForBERT, masked_perplexity,
    MaskedPenalizedSparseCategoricalCrossentropy)

import keras_metrics as km
from keras_trans_mask import RemoveMask, RestoreMask

Using TensorFlow backend.


In [2]:
from models import transformer_bert_model
from bpe import BPEEncoder
from utils import *

### load data

In [3]:
train_emb, train_emb_api, train_fam_ans, train_rep_ans = pickle.load(open('data/tree-rep-profiles-partial/TRAIN_vec.pkl','rb'))
valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans = pickle.load(open('data/tree-rep-profiles-partial/DEV_vec.pkl','rb'))
test_emb, test_emb_api,test_fam_ans,test_rep_ans = pickle.load(open('data/tree-rep-profiles-partial/TEST_vec.pkl','rb'))
# print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_fam_ans.shape,train_rep_ans.shape)
# print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_fam_ans.shape,valid_rep_ans.shape)
train_rep_ans = np.expand_dims(train_rep_ans,axis=-1)
valid_rep_ans = np.expand_dims(valid_rep_ans,axis=-1)
test_rep_ans = np.expand_dims(test_rep_ans,axis=-1)
print('test of sent2vec vector:',test_emb.shape,test_emb_api.shape,test_fam_ans.shape,test_rep_ans.shape)

test of sent2vec vector: (272, 213, 700) (272, 213) (272, 87) (272, 213, 1)


In [4]:
def _shuffle(X, X2 ,X3,X4):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], X2[randomize],X3[randomize],X4[randomize])

In [5]:
train_emb, train_emb_api, train_fam_ans, train_rep_ans = _shuffle(train_emb, train_emb_api, train_fam_ans, train_rep_ans)
valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans = _shuffle(valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans)
# test_emb, test_emb_api,test_fam_ans,test_rep_ans  = _shuffle(test_emb,test_emb_api,test_fam_ans,test_rep_ans)
print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_fam_ans.shape,train_rep_ans.shape)
print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_fam_ans.shape,valid_rep_ans.shape)
# print('test of sent2vec vector:',test_emb.shape,test_emb_api.shape,test_fam_ans.shape,test_rep_ans.shape)

train of sent2vec vector: (11925, 213, 700) (11925, 213) (11925, 87) (11925, 213, 1)
valid of sent2vec vector: (336, 213, 700) (336, 213) (336, 87) (336, 213, 1)


feature scaling

In [6]:
# scale = 'no'

def scaling(trainX,validX,testX,scale='min_max'):
#     if scale == 'min_max':
    max_value = max([np.max(trainX) , np.max(validX),np.max(testX)])
    min_value = min([np.min(trainX),np.min(validX),np.min(testX)])

    trainX = (trainX - min_value) / (max_value - min_value)
    validX = (validX - min_value) / (max_value - min_value )
    testX = (testX - min_value) / (max_value - min_value )
    print(np.max(trainX),np.max(validX))
    return trainX,validX,testX , max_value , min_value

train_emb,valid_emb,test_emb , max_value,min_value = scaling(train_emb,valid_emb,test_emb)   
print(valid_emb.shape)

1.0 1.0
(336, 213, 700)


many dense output: (MML)

In [7]:
'''def change2MML(np_array):
    temp = np.expand_dims(np_array,axis=-1)
    temp = np.swapaxes(temp,0,1)
    return temp
train_fam_ans = change2MML(train_fam_ans)
valid_fam_ans = change2MML(valid_fam_ans)
# test_fam_ans = change2MML(test_fam_ans)
print(valid_fam_ans.shape)'''

'def change2MML(np_array):\n    temp = np.expand_dims(np_array,axis=-1)\n    temp = np.swapaxes(temp,0,1)\n    return temp\ntrain_fam_ans = change2MML(train_fam_ans)\nvalid_fam_ans = change2MML(valid_fam_ans)\n# test_fam_ans = change2MML(test_fam_ans)\nprint(valid_fam_ans.shape)'

In [8]:
# temp = [[1,0],[0,0],[1,1]]
# temp = np.array(temp)
# temp = np.expand_dims(temp,axis=-1)
# print(temp.shape)
# # temp = temp.reshape(2,3,1)
# # temp = np.swapaxes(temp,0,1)
# temp

In [9]:
# randomize = np.arange(len(temp))
# np.random.shuffle(randomize)
# temp[randomize]

## Model Hyper-parameters

In [4]:
# 模型參數
l2_reg_penalty = 1e-4#1e-4
embedding_dropout = 0.6
transformer_dropout = 0.1
use_universal_transformer = True #true=>ACT

transformer_depth = 2
vocabulary_size = 26 #api name種類
max_seq_length = test_emb.shape[1] # profile最大長度
word_embedding_size = test_emb.shape[2]#被除數，跟Sen2Vec最終維度相同
num_heads = 2#除數，要整除
fam_num = test_fam_ans.shape[1]#test_fam_ans.shape[1] MML
batch_size = 256 #128

CONFIDENCE_PENALTY = 0.1


### model input

In [5]:
sent_ids1 = Input(shape=(max_seq_length,), dtype='int32', name='sent_ids') # 輸入的api funvtion name ID
sent_ids = Masking(mask_value=0)(sent_ids1)
#shape=(max_seq_length,),batch_shape=(batch_size,max_seq_length)
sentemb1 = Input(shape=(max_seq_length,word_embedding_size),name='sent_emb') #輸入Sent2Vec的embeeding (長度,維度大小)
sentemb = Masking(mask_value=0)(sentemb1)
#shape=(max_seq_length,word_embedding_size),,batch_shape=(batch_size,max_seq_length,word_embedding_size)

In [6]:
# model function
l2_regularizer = (regularizers.l2(l2_reg_penalty) if l2_reg_penalty else None)

add_segment_layer = Add(name='add_segment')

multiply_embedding_layer = Multiply(name='mul_emb')

In [7]:
# embedding function
embedding_layer = ReusableEmbedding( # 希望可以跟model一起學的embedding
        vocabulary_size, word_embedding_size, #emb size
        input_length=max_seq_length,
        name='bpe_embeddings',
        # Regularization is based on paper "A Comparative Study on
        # Regularization Strategies for Embedding-based Neural Networks"
        # https://arxiv.org/pdf/1508.03721.pdf
        embeddings_regularizer=l2_regularizer,trainable=True)
# 一個sentence一個ID
# segment_embedding_layer = Embedding(vocabulary_size ,word_embedding_size,weights=[emb_matrix]
#                                    input_length=max_seq_length , name='segment_embeddings')  #利用pre-train sent2vec vector，跟前面的加在一起

coordinate_embedding_layer = TransformerCoordinateEmbedding(
        transformer_depth , name='coordinate_embedding')

In [8]:
next_step_input_emb, embedding_matrix = embedding_layer(sent_ids)
# segment_embeddings = segment_embedding_layer(sent_ids) # 需增加pretrained Sent2Vec的embedding matrix
segment_embeddings = sentemb

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
output_layer = TiedOutputEmbedding(
        projection_regularizer=l2_regularizer,
        projection_dropout=embedding_dropout,
        name='word_prediction_logits')

output_softmax_layer = Softmax(name='fam_predictions') # 2nf stage

### transformer

In [18]:
next_step_input1 = RemoveMask()(segment_embeddings)
next_step_input = coordinate_embedding_layer(next_step_input1, step=0) #next_step_input_emb
next_step_input= RestoreMask()([next_step_input,next_step_input1])
next_step_input = add_segment_layer([next_step_input, next_step_input_emb]) #segment_embeddings
# next_step_input = Concatenate()([next_step_input,next_step_input_emb])
for i in range(transformer_depth):
    next_step_input1 = RemoveMask()(next_step_input)
    next_step_input = (
        TransformerBlock(
            name='transformer' + str(i), num_heads=num_heads,
            residual_dropout=transformer_dropout,
            attention_dropout=transformer_dropout,
            use_masking=False,  # Allow bi-directional attention
            vanilla_wiring=True)
        (next_step_input1))
    next_step_input = RestoreMask()([next_step_input,next_step_input1])

In [17]:
# act_layer = TransformerACT(
#             name='adaptive_computation_time')

# transformer_block = TransformerBlock(
#             name='transformer', num_heads=num_heads,
#             residual_dropout=transformer_dropout,
#             attention_dropout=transformer_dropout,
#             # Allow bi-directional attention
#             use_masking=False)

# act_output = next_step_input

# for i in range(transformer_depth):
#             next_step_input = coordinate_embedding_layer(
#                 next_step_input, step=i)
#             next_step_input = add_segment_layer(
#                 [next_step_input, segment_embeddings])
#             next_step_input1 = RemoveMask()(next_step_input)
#             next_step_input2 = transformer_block(next_step_input1)
# #             next_step_input = RestoreMask()([next_step_input2,next_step_input1])
#             next_step_input, act_output = act_layer(next_step_input2)
#             next_step_input = RestoreMask()([next_step_input,next_step_input1])

# act_layer.finalize()
# next_step_input = act_output
# next_step_input = RestoreMask()([next_step_input,act_output])

In [11]:
# trm_output = next_step_input #output_layer([next_step_input, embedding_matrix]) ?
family_input = output_layer([next_step_input1, embedding_matrix]) #embedding_matrix，
family_input1 = RestoreMask()([family_input,next_step_input1]) #MASK
next_step_input = BatchNormalization()(next_step_input)
class_prediction = (
        Dense(1, name='0_1_prediction', activation=hard_sigmoid) #sigmoid?hard_sigmoid
        (next_step_input)) #輸出重要性，如果要用softmax就會變成Dense(2)

family_input = multiply_embedding_layer([class_prediction,next_step_input]) # 2nd stage next_step_input/family_input
# family_input = BatchNormalization()(family_input)
family_prediction = Bidirectional(LSTM(fam_num,kernel_regularizer=l2_regularizer,
                                            recurrent_regularizer=l2_regularizer,return_sequences=True,
                                           dropout=transformer_dropout, #stateful=True,
                                            recurrent_dropout=transformer_dropout,name='fam_feature'))(family_input)
family_prediction = Concatenate()([family_prediction,family_input])
family_prediction = BatchNormalization()(family_prediction)
family_prediction = (
        Bidirectional(GRU(fam_num,name='fam_clf'))(family_prediction) #int((word_embedding_size+vocabulary_size)/4)
) # 2nd stage，變成bidirectional加dense?，變成兩層?加L2
family_prediction = Dense(fam_num,activation='sigmoid',name='family')(family_prediction)
#MML
'''out={}
for i in range(fam_num):
    out[i] = Dense(1,activation='sigmoid',name='family'+str(i))(family_prediction)'''
# family_prediction = output_softmax_layer(family_prediction) # 2nd stage

"out={}\nfor i in range(fam_num):\n    out[i] = Dense(1,activation='sigmoid',name='family'+str(i))(family_prediction)"

In [12]:
# model = Model(inputs = [sent_ids,sentemb],outputs=[class_prediction]) #1st stage
#MML
'''outs = []
for i in range(fam_num):
    outs.append(out[i])
model = Model(inputs = [sent_ids1,sentemb1],outputs=[class_prediction]+outs) # 1st stage fine-tune'''
# model = Model(inputs = [sent_ids1,sentemb1],outputs=[class_prediction,family_prediction])
model = Model(inputs = [sent_ids1,sentemb1],outputs=[family_prediction]) #2nd stage
model = multi_gpu_model(model , gpus=3) #如果不能儲存multiGPU model，只能single GPU，那LSTM就改成stateful=True
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_emb (InputLayer)           (None, 10, 100)      0                                            
__________________________________________________________________________________________________
masking_2 (Masking)             (None, 10, 100)      0           sent_emb[0][0]                   
__________________________________________________________________________________________________
remove_mask_1 (RemoveMask)      (None, 10, 100)      0           masking_2[0][0]                  
__________________________________________________________________________________________________
sent_ids (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
coordinate

### original model archietecture

In [20]:
try:
    single_model = model.layers[-3]
    single_model.summary()
except:
    pass

* MTL:
https://datascience.stackexchange.com/questions/28827/multi-task-learning-architecture-for-multi-label-classification
* customized loss
* f1_loss
* feature scaling
* concate api vector
* imbalance learn

### Metrics & loss & save path

In [21]:
def full_multi_label_metric(y_true, y_pred):
    comp = K.equal(y_true, K.round(y_pred))
    return K.cast(K.all(comp, axis=-1), K.floatx())
def f1_metric(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)
# from sklearn.metrics import f1_score
# def f1_sk(y_true,y_pred):
#     score = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')
#     return score
# model.load_weights('./model/trm_encoder/1stStage_87fam_0524.h5')
# 訓練參數
los = [losses.binary_crossentropy,f1_loss] # 1st stage. 
#SINGLE
los = [binary_focal_loss(gamma=2., alpha=.25)]
#MML
'''los = []
for i in range(fam_num):
    los.append(binary_focal_loss(alpha=.25, gamma=2))
los = [losses.binary_crossentropy] + los'''


metric = {'RasMMA': 'acc','family': f1} # 1st stage. km.f1_score()
#SINGLE
metric = {'family': f1_metric}
#MML
'''metrics = []
for i in range(fam_num+1):
    metrics.append('acc')
# metrics = {}
# metrics['RasMMA'] = 'acc'
# for i in range(fam_num):
#     metrics['fam'+str(i)]='acc'
metric = metrics'''


loss_weight = [0.01,1] #stage1 0.95,0.05  #1st stage # 2nd stage [0.01,0.99]
#SINGLE
loss_weight = [1]
#MML
'''loss_weight = []
for i in range(fam_num):
    loss_weight.append(0.95)
loss_weight = [0.05] + loss_weight'''

learning_rate = 2e-4 # 2nd stage: 1e-4 @1st:2e-4
batch_size = batch_size #32 #128

num_epochs = 1000
patien = 50

model_save_path = './model/trm_encoder/1stStage_87fam_0524.h5'
tensorboard_log_path = './logs/'+ model_save_path.split('/')[-1].split('.')[0]

print valid loss metric only: https://stackoverflow.com/questions/47774931/keras-print-metrics-only-for-validation-data

In [22]:
optimizer = optimizers.Adam(
            lr=learning_rate, beta_1=0.9, beta_2=0.999, clipnorm=1., amsgrad=False)

lr_scheduler = callbacks.LearningRateScheduler(
        CosineLRSchedule(lr_high=learning_rate, lr_low=1e-8,
                         initial_period=num_epochs),
        verbose=1)

model.compile(
            optimizer,
            loss=los,
            metrics=metric ,loss_weights=loss_weight)#{'word_predictions': masked_perplexity})

In [23]:
# save best, early stopping, 2 models ens weight:(best=0.8,last=0.2)
history = History()
stop_nan = callbacks.TerminateOnNaN()
model_callbacks = [
        callbacks.ModelCheckpoint(
            model_save_path,
            monitor='val_loss',mode='min' ,save_best_only=True, verbose=1,save_weights_only=True),
            EarlyStopping(patience=patien,monitor='val_loss',verbose=1,mode='min'),
        lr_scheduler, history,stop_nan
    ]
model_callbacks.append(callbacks.TensorBoard(tensorboard_log_path))

### Fit Generator

In [24]:
def training_batch(batch_size, X_train1, X_train2 , Y_train1, Y_train2):
    '''
    X_train1 = sent_ids: shape為(N, max_seq_length)
    X_train2 = sentemb: shape為(N,max_seq_length, word_embedding_size)
    Y_train1 = class_prediction: shape為(N, max_seq_length, 1)
    Y_train2 = family_prediction(stage2): shape為(N, fam_num)
    '''
    idx = np.arange(len(X_train1))
    np.random.shuffle(idx)
#     if Y_train2 == None:
#         print('Input shape of X1,X2,Y1,Y2:',X_train1.shape,X_train2.shape,Y_train1.shape)
#         while True:
#             for i in idx:
#                 train_X1 = X_train1[idx[i]:idx[i]+batch_size]
#                 train_X2 = X_train1[idx[i]:idx[i]+batch_size]
#                 train_Y1 = X_train1[idx[i]:idx[i]+batch_size]
#                 yield train_X1,train_X2,train_Y1
#                 if i == idx[-1]:
#                     idx = np.arange(len(X_train1))
#                     np.random.shuffle(idx)
#                     break
#     print('Input shape of X1,X2,Y1,Y2:',X_train1.shape,X_train2.shape,Y_train1.shape,Y_train2.shape) #(all data,profile length,api )

    while True:
        for i in idx:
            train_X1 = X_train1[idx[i]:idx[i]+batch_size]
            train_X2 = X_train2[idx[i]:idx[i]+batch_size]
            train_Y1 = Y_train1[idx[i]:idx[i]+batch_size]
            train_Y2 = Y_train2[idx[i]:idx[i]+batch_size]
#             yield ([train_X1,train_X2],[train_Y1,train_Y2]) #ori
            yield ([train_X1,train_X2],[train_Y2])
            if i == idx[-1]:
                idx = np.arange(len(X_train1))
                np.random.shuffle(idx)
                break
            
#     data_size = X_train.shape[0]
#     ep = data_size / batch_size

In [25]:
#MTM
'''def training_batch(batch_size, X_train1, X_train2 , Y_train1, Y_train2):

    idx = np.arange(len(X_train1))
    np.random.shuffle(idx)
    
    fam_dict = {}
    for i in range(fam_num):
        fam_dict[i] = Y_train2[i]
    while True:
        for i in idx:
            train_X1 = X_train1[idx[i]:idx[i]+batch_size]
            train_X2 = X_train2[idx[i]:idx[i]+batch_size]
            train_Y1 = Y_train1[idx[i]:idx[i]+batch_size]
            train_Y2 = []
            for famid in range(fam_num):
                temp = fam_dict[famid][idx[i]:idx[i]+batch_size]
                train_Y2.append(temp)
#             train_Y2 = Y_train2[idx[i]:idx[i]+batch_size]
            trainY = [train_Y1]+train_Y2
#             print(len(trainY))
            yield ([train_X1,train_X2],trainY)
            if i == idx[-1]:
                idx = np.arange(len(X_train1))
                np.random.shuffle(idx)
                break

# valid_fam_ans.shape
valid_Y2 = []
for i in range(fam_num):
    valid_Y2.append(valid_fam_ans[i])'''
# valid_fam_ans = valid_Y2

'def training_batch(batch_size, X_train1, X_train2 , Y_train1, Y_train2):\n\n    idx = np.arange(len(X_train1))\n    np.random.shuffle(idx)\n    \n    fam_dict = {}\n    for i in range(fam_num):\n        fam_dict[i] = Y_train2[i]\n    while True:\n        for i in idx:\n            train_X1 = X_train1[idx[i]:idx[i]+batch_size]\n            train_X2 = X_train2[idx[i]:idx[i]+batch_size]\n            train_Y1 = Y_train1[idx[i]:idx[i]+batch_size]\n            train_Y2 = []\n            for famid in range(fam_num):\n                temp = fam_dict[famid][idx[i]:idx[i]+batch_size]\n                train_Y2.append(temp)\n#             train_Y2 = Y_train2[idx[i]:idx[i]+batch_size]\n            trainY = [train_Y1]+train_Y2\n#             print(len(trainY))\n            yield ([train_X1,train_X2],trainY)\n            if i == idx[-1]:\n                idx = np.arange(len(X_train1))\n                np.random.shuffle(idx)\n                break\n\n# valid_fam_ans.shape\nvalid_Y2 = []\nfor i in ran

In [26]:
H = model.fit_generator(generator=training_batch(batch_size=batch_size,X_train1=train_emb_api,X_train2=train_emb ,
                                             Y_train1=train_rep_ans,Y_train2=train_fam_ans) #Y_train2
                    , steps_per_epoch=int(np.ceil(len(train_emb_api)/batch_size)) ,
                    epochs=num_epochs,callbacks=model_callbacks
#                    ,validation_data= ([valid_emb_api,valid_emb], [valid_rep_ans,valid_fam_ans]) #ori
                   ,validation_data= ([valid_emb_api,valid_emb], [valid_fam_ans]) #ori
#                    ,validation_data= ([valid_emb_api,valid_emb], [valid_rep_ans]+valid_Y2) #validY2
                    ,max_queue_size=10  #,class_weight='auto'
                    ,workers=10,use_multiprocessing=True   
                   ,shuffle=True,verbose=1)
model.save(model_save_path+"_all.h5")
#1st:train 0_1_prediction=0.14XX

Instructions for updating:
Use tf.cast instead.


/home/leoqaz12/.local/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/1000

Epoch 00001: LearningRateScheduler setting learning rate to 0.0002.
 4/47 [=>............................] - ETA: 3:40 - loss: 2589.8497 - f1_metric: 0.0319

Process ForkPoolWorker-1:
Process ForkPoolWorker-6:
Process ForkPoolWorker-2:
Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-8:
Process ForkPoolWorker-5:
Process ForkPoolWorker-10:
Process ForkPoolWorker-7:
Process ForkPoolWorker-9:
Traceback (most recent call last):
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/leoqaz12/anaconda3/lib/python3.6/mult

KeyboardInterrupt: 

### plot result

In [ ]:
# plot the training loss and accuracy
import matplotlib as plt
plt.style.use("ggplot")
plt.figure()
N = num_epochs
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig(args["plot"])

## evaluate

In [ ]:
model.load_weights('./model/trm_encoder/1stStage_87fam_0524.h5')
# score = model.evaluate([valid_emb_api,valid_emb], [valid_rep_ans]+valid_Y2)
print(len(test_emb_api))
# ans = model.predict([test_emb_api,test_emb])
ans = model.predict([valid_emb_api,valid_emb])
len(ans)

In [ ]:
predict_fam = np.array(ans[1:])
predict_fam = np.swapaxes(predict_fam,0,1)
valid_fam_ans1 = np.swapaxes(valid_fam_ans,0,1)
predict_fam.shape , valid_fam_ans1.shape

In [ ]:
from sklearn.metrics import f1_score
# y_true = np.squeeze(test_fam_ans)
y_true = np.squeeze(valid_fam_ans1)
y_pred = np.squeeze(predict_fam)
final_ans = []
for sample in y_pred:
    sample_ans = []
    for value in sample:
        if value < 0.25:
            sample_ans.append(0)
        else:
            sample_ans.append(1)
    final_ans.append(sample_ans)
final_ans = np.array(final_ans)
print(final_ans.shape , sum(final_ans[0]))

In [ ]:
# test_fam_ans = np.swapaxes(test_fam_ans,0,1)
# test_fam_ans.shape
# y_true = np.array([[1,0,0,0], [1,1,0,0], [1,1,1,1]])
# y_pred = np.array([[1,0,0,0], [1,1,1,0], [1,1,1,1]])
print(y_true.shape , final_ans.shape)
recall = recall_score(y_true=y_true, y_pred=final_ans, average='micro')
precision = precision_score(y_true=y_true, y_pred=final_ans, average='micro')
f1 = f1_score(y_true=y_true, y_pred=final_ans, average='micro')
recall ,precision, f1

In [ ]:
recall

In [ ]:
y_true[0] , final_ans[0]